### 네이버 영화 리뷰를 크롤링하여 데이터프레임으로 변환

In [10]:
import requests
from  bs4 import BeautifulSoup
import time
import csv

In [16]:
from re import findall
reviews = []
review_data = []
need_reviews_cnt = 1000

# page를 1부터 1씩 증가하며 URL을 다음 페이지로 바꿈
for page in range(1, 100):
  url = f'https://movie.naver.com/movie/point/af/list.naver?&page={page}'
  #get : request로 url의 html문서의 내용 요청
  html = requests.get(url)
  # print(html)
  #html을 받아온 문서를 .content로 지정 후 soup객체로 변환
  soup = BeautifulSoup(html.content, 'html.parser')
  # print(soup)
  #find_all : 지정한 태그의 내용을 모두 찾아 리스트로 반환
  reviews = soup.find_all("td", {"class":"title"})

  # 한 페이지의 립 리스트의 리뷰를 하나씩 보면서 데이터 추출
  for review in reviews:
    sentence = review.find('a', {'class':'report'}).get('onclick').split("', ")[2]
    # 만약 리뷰 내용이 비어있다면 데이터를 사용하지 않음
    if sentence != "":
      movie = review.find('a', {'class':'movie color_b'}).get_text()
      score = review.find('em').get_text()
      review_data.append([movie, sentence, int(score)])
      need_reviews_cnt -= 1
  # 현재까지 수집된 리뷰가 목표 수집 리뷰보다 많아진 경우 크롤링 중지
  if need_reviews_cnt < 0:
    break
  # 다음 페이지를 조회하기 전 0.5초 시간 차를 두기
  time.sleep(0.5)

columns_name = ['movie', 'sentence', 'score']
with open('samples.csv', 'w', newline='', encoding='utf-8') as f:
  write = csv.writer(f)
  write.writerow(columns_name)
  write.writerows(review_data)

In [17]:
import pandas as pd
data = pd.read_csv('samples.csv')
data

,movie,sentence,score
0,탄생,'가톨릭신자라면 꼭 봐야하는 영화 후반부 부터 감동이 계속 가슴에 어머니에게 영성체...,10
1,탄생,'지인과 가족과 이렇게 두번 관람했습니다. 매번 같은 장면들에서 눈물이 흐르더라구요...,10
2,싱크홀,'내 인생영화….진짜 몰입 너무 잘됩니다ㅠㅠ,10
3,압꾸정,'그냥 마동석이랑 정경호만 개그치는거 재미있게보고왔음ㅋㅋㅋ 무슨스토리인지모르겠고 끝...,10
4,올빼미,',9
...,...,...,...
985,탄생,'너무 감동적인 영화에요. 강추에요.,10
986,멋진 하루,'어른을 위한 동화같은하정우에하정우를 위한 영화,10
987,올빼미,'올해 한국영화 중에서 가장 좋았어요 시작부터 왠지 모르게 처연하고 보는내내 슬펐습...,10
988,올빼미,'소현세자역 김성철 배우 연기 짱이에용,10


In [31]:
# 영화별로 그룹을 지어 평점 평균을 구하시오
a = data.groupby('movie')['score'].sort_values(by='score')
a

AttributeError: ignored